# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [12]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

[program['id'] for program in data['programs']]

['qasm3-runner', 'sampler', 'estimator', 'circuit-runner']

## Get temporary Access token from Auth API via API Token

In [13]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [14]:
url_backends = 'https://runtime-us-east.quantum-computing.ibm.com/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['simulator_stabilizer', 'simulator_extended_stabilizer', 'ibmq_qasm_simulator', 'ibm_nairobi', 'ibm_lagos'] ...


## Run a job

### set up qasm circuit

In [15]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [17]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibm_kyiv',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "start_session": "True", #set to False if you just need to run a single job
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cn9pzc51mh9g008yf8y0","backend":"ibm_kyiv","session_id":"cn9pzc51mh9g008yf8y0"}


In [18]:
session_id=response.json()['session_id']
session_id

'cn9pzc51mh9g008yf8y0'

### Run follow-up jobs in the same Session

In [19]:
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibmq_qasm_simulator',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": session_id,
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cl9mvg16mcdhud5g0v3g","backend":"ibmq_qasm_simulator","session_id":"cn9pzc51mh9g008yf8y0"}


### Query Websocket for live job update and close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. So we will listen for a status change of the last job via websocket and close the Session when done.

In [20]:
import asyncio
import websockets

with open('token') as file:
    api_key=file.read()

async def receive_status_updates(api_key):
    uri = "wss://runtime-us-east.quantum-computing.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}'}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                break

await receive_status_updates(api_key)

#subsequently closing Session
status_result = requests.get(url+'/'+job_id,headers=headers)
sessionsurl = 'https://runtime-us-east.quantum-computing.ibm.com/sessions/'+session_id+'/close'

if status_result.json()['state']['status']=='Completed':
   response = requests.delete(sessionsurl,headers=headers)
else:
   print('waiting for job to complete before closing Session '+session_id)

if response.status_code==204:
    print('Session '+session_id+' was closed')

Received message: Job is not running. Job status is 'Completed', reason is ''.  Check '/jobs/cl9mvg16mcdhud5g0v3g/results' for results
WebSocket connection closed.
Session cn9pzc51mh9g008yf8y0 was closed


### Get results

In [21]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'1': 1.0}, {'1': 1.0}, {'1': 1.0}, {'1': 1.0}, {'1': 1.0}],
 'metadata': [{'shots': 1024,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.07012054900405928},
  {'shots': 1024,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.049093275010818616},
  {'shots': 1024,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.049420405994169414},
  {'shots': 1024,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.043661167001118883},
  {'shots': 1024,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.04399759098305367}]}

## Invalidate Token

In [22]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [23]:
rl = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibmq_qasm_simulator',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "params": {
        "circuits": [qasm_string]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(response.text,"\n")
    print(f"Error: {response.status_code}")

{"errors":[{"code":1201,"message":"Invalid credentials.","solution":"Verify your credentials and try again.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]} 

Error: 401
